# Preliminary Classification of Unlabelled Tweets

* Complete and improved tweet tweet classifcation can be found in /notebooks/twitter_data_exploration_400k_tweets.ipynb
* This notebook classified the smaller 164K tweet dataset while the projects feasability was being assessed.
 * Another notebook will be created to classify the complete dataset
* Below, a Logistic Regression classifier has been trainied on SemEval2017 Task A Golden Dataset + additional positive :)  and negative  :(  tweets, collected from twitter. This will serve as a baseline model.
* Finalized versions of the functions below now exist in functions.py.
* [VADER](https://github.com/cjhutto/vaderSentiment) will also be utilized for classification and compared to the baseline models.

In [37]:
import re
import nltk
import string
import pickle
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
stop_words = nltk.corpus.stopwords.words('english')
from sklearn.feature_extraction.text import TfidfVectorizer
pd.set_option('display.max_colwidth', None)

In [38]:
# 164k Tweets - Prelimiinary Dataset for feasability paper - Hydrated with Twarc
#df = pd.read_json('/Volumes/My Passport/Tweets/bc_tweets.jsonl', lines=True)

# Full Tweet Dataset - Aug 14th to Present
COL_NAMES = ['id_str','created_at','full_text']
df = pd.read_csv('/Users/lclark/Documents/GitHub/bcpolidata/bcpoli.csv', names=COL_NAMES)

In [39]:
raw = df.copy()
raw = raw.rename(columns={'id_str': 'ids','full_text':'text'})
raw.set_index('ids', inplace=True)
raw = raw[['created_at','text']]
raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 382918 entries, 1296960190068056064 to 1327038530586079238
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   created_at  382918 non-null  object
 1   text        382918 non-null  object
dtypes: object(2)
memory usage: 8.8+ MB


## Preprocess Tweets

In [43]:
def emoji_stringer(text):
    # Positive Emoji - Smile, Laugh, Wink,Love
    text = ' '.join(re.sub('(:\s?\)|:-\)|;\)|\(\s?:|\(-:|:\’\))','emopos',text).split())
    text = ' '.join(re.sub('(:\s?D|:-D|x-?D|X-?D)','emopos',text).split()) 
    text = ' '.join(re.sub('(<3|:\*)','emopos',text).split()) 
    # Negative Emoji - Sad, Cry
    text = ' '.join(re.sub('(:\s?\(|:-\(|:\||\)\s?:|\)-:)','emoneg',text).split())
    text = ' '.join(re.sub('(:,\(|:\’\(|:"\()','emoneg',text).split())
    return text

raw['text']=raw['text'].apply(str)
raw['emo_features'] = raw['text'].apply(lambda x: emoji_stringer(x))

In [51]:
# Preprocess Tweets - lowercase, URLS, tokens and punctuation

def preprocess(text):
    text = text.lower()
    text = ' '.join(re.sub("((www\.[\S]+)|(https?://[\S]+))","",text).split())
    text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",text).split())
    text = ' '.join(re.sub("^\n","",text).split()) # Handles Tweepy Case - Not required for processed twarc jsonl
    text = ' '.join(re.sub("^brt","",text).split()) # Handles Tweepy Case - Not required for processed twarc jsonl
    text = ' '.join(re.sub("^rt","",text).split()) 
    punc = ''.join([char for char in text if char not in string.punctuation])
    tokens = word_tokenize(punc)
    stops = [word for word in tokens if word not in stop_words]
    strings = (' ').join(stops)
    strings = ' '.join(re.sub("xe2x80xa6","",text).split()) # Handles Tweepy encoding Case -  Not required for processed twarc jsonl
    return strings

raw['full_text_clean'] = raw['emo_features'].apply(lambda x: preprocess(x))

In [65]:
raw.head()

,created_at,text,emo_features,full_text_clean,labels,nb_labels,lr_g_labels
ids,,,,,,,
1296960190068056064,2020-08-21 23:59:40,b'RT @LizaYuzda: BC #covid19 update Aug 21\n2 new deaths - 202 \n90 new\nActive 824 (798)\nVCH 1579 (1488)\nFH 2572 (2493)\nIsland 160 (158)\nInteri\xe2\x80\xa6',b'RT @LizaYuzda: BC #covid19 update Aug 21\n2 new deaths - 202 \n90 new\nActive 824 (798)\nVCH 1579 (1488)\nFH 2572 (2493)\nIsland 160 (158)\nInteri\xe2\x80\xa6',bc covid19 update aug 21n2 new deaths 202 n90 newnactive 824 798nvch 1579 1488nfh 2572 2493nisland 160 158ninteri,4,2,4
1296960183386554368,2020-08-21 23:59:38,"b""RT @realreporter: #bcpoli As for child infections the lie has been children don't get readily infected. \n2 days ago 0-10 infections were 11\xe2\x80\xa6""","b""RT @realreporter: #bcpoli As for child infections the lie has been children don't get readily infected. \n2 days ago 0-10 infections were 11\xe2\x80\xa6""",bcpoli as for child infections the lie has been children dont get readily infected n2 days ago 010 infections were 11,2,4,4
1296960130785804288,2020-08-21 23:59:26,"b'RT @photowarrior: VicPD correcting their previous error, but ""correct"" stats don\'t change the fact that policing is no solution to poverty,\xe2\x80\xa6'","b'RT @photowarrior: VicPD correcting their previous error, but ""correct"" stats don\'t change the fact that policing is no solution to poverty,\xe2\x80\xa6'",vicpd correcting their previous error but correct stats dont change the fact that policing is no solution to poverty,4,4,0
1296960084421914624,2020-08-21 23:59:15,b'RT @bychrischeung: Did you know the Himalayan blackberry is an invasive species?\n\nMy big berry history stretches thousands of years and inc\xe2\x80\xa6',b'RT @bychrischeungemoposid you know the Himalayan blackberry is an invasive species?\n\nMy big berry history stretches thousands of years and inc\xe2\x80\xa6',you know the himalayan blackberry is an invasive speciesnnmy big berry history stretches thousands of years and inc,4,4,2
1296959921305448448,2020-08-21 23:58:36,b'RT @RobPaynterSD61: So apparently even thoughtful countries that have been diligent in addressing COVID are being hit hard after reopening\xe2\x80\xa6',b'RT @RobPaynterSD61: So apparently even thoughtful countries that have been diligent in addressing COVID are being hit hard after reopening\xe2\x80\xa6',so apparently even thoughtful countries that have been diligent in addressing covid are being hit hard after reopening,2,2,4


In [53]:
vectorizer = TfidfVectorizer(use_idf=True, lowercase=True, ngram_range=(1,2), max_features=40000)

X = vectorizer.fit_transform(raw.full_text_clean)

## Import Models

In [54]:
lr_model = pickle.load(open('tweet_40kfeat_LogReg_CLF_3C_86p_model.sav', 'rb'))
lr_grid_model = pickle.load(open('tweet_40kfeat_LR_GridCV_3C_89p_model.sav','rb'))
nb_model = pickle.load(open('tweet_40kfeat_NB_CLF_3C_83p_model.sav', 'rb'))

In [55]:
raw['labels'] = lr_model.predict(X)

In [56]:
raw['nb_labels'] =  nb_model.predict(X)

In [57]:
raw['lr_g_labels'] = lr_grid_model.predict(X)

In [40]:
#logreg_pred = lr_model.predict(X)

In [58]:
raw.labels.value_counts()

2    203032
4    146784
0     33102
Name: labels, dtype: int64

In [59]:
raw.nb_labels.value_counts()

2    197860
4    138957
0     46101
Name: nb_labels, dtype: int64

In [60]:
raw.lr_g_labels.value_counts()

2    202336
0    110271
4     70311
Name: lr_g_labels, dtype: int64

In [ ]:
# Below is on 164k tweets

In [41]:
raw.labels.value_counts()

2    80349
4    72920
0    11279
Name: labels, dtype: int64

In [42]:
raw.nb_labels.value_counts()

4    75986
2    64915
0    23647
Name: nb_labels, dtype: int64

In [87]:
raw.lr_g_labels.value_counts()

2    70092
0    53192
4    41264
Name: lr_g_labels, dtype: int64

In [61]:
export = raw[['created_at','full_text_clean','labels','nb_labels','lr_g_labels']]#.reset_index()
#export.resample('D', on='created_at').mean()
#export.ids = export.ids.astype('str')
#export.resample('D', on='created_at').mean()#.groupby(['ids']).mean()
export = export.reset_index()
export.to_pickle('380_tweepy_classified_tweets.pkl')

In [ ]:
# Check mean values of labels with data is scaled up to full datset
# As can be seen, the mean values are similar, demonstrating reasonable levels of consistency in the baseline models

In [62]:
raw.labels.mean()

2.593766811693365

In [63]:
raw.nb_labels.mean()

2.484991564773659

In [64]:
raw.lr_g_labels.mean()

1.7912869073796478

In [ ]:
# Below is on 164k tweets

In [52]:
raw.labels.mean()

2.5476

In [53]:
raw.nb_labels.mean()

2.6942

In [66]:
raw.lr_g_labels.mean()

1.7764